<font size="8" color ="Brown"><center>
# Lecture 6, Video Coding, Motion Compensation
        
<center></font>
<br>

<p style="line-height:1.5">
<font size="6">
    
Last time we saw 2 error measures for the motion estimation. We now have an optimization task: minimize the error with our motion vectors. Mathematically we can write this task as

$$ { \binom{{v_{min} = argmin}}{  v} } ⁡  {{MSE  (  v  )} }$$

<br>
where $v_{min}$ is our motion vector. “argmin” means we minimize the MSE with v, and the result is the argument at which the MSE becomes the minimum.<br>
How do we **search** the image for the best motion vector $v_{min}$? A simple, but computationally inefficient way would be to simply scan a search area (it can be an area smaller than the entire picture to reduce complexity) in raster order,<br><br>
![Lecture6b-1.PNG](Img-Lecture6b\Lecture6b-1.PNG)
<br><br>

If we are searching for motion vectors with pixel accuracy, we need to advance our search block pixel-wise. This is also called **full search** motion estimation, which leads to a relatively **high computational complexity**.<br> 
To **reduce complexity**, we can make the assumption that the motion vector is close to the origin with high probability (small motion). Hence we can modify this search approach by starting in the origin and have a spiral wise search path. We can then stop this search if we have some stopping criteria (e.g. reaching a local minimum or a certain threshold of the error),<br>

![Lecture6b-2.PNG](Img-Lecture6b\Lecture6b-2.PNG)
<br>

This is a spiral order. But this approach has the problem that we can easily get stuck in a **local minimum**. To avoid this problem we can apply a hierarchical search order, for instance the so-called three step search (see Richardson, Video Codec Design), where we start with a coarse search of our search area, and then refine our search near the minimum of the coarser search. We can do this in 3 steps,<br>

![Lecture6b-3.PNG](Img-Lecture6b\Lecture6b-3.PNG)
<br>

The hierarchical three step search starts with a coarse search, which means we advance the block by more than one pixel, in the example by several pixels. In this way we get a coarse motion vector to the minimum of this first step, with an accuracy of a few pixels. In the second step, we set this minimum as the new center of our search, and obtain a motion vector with increased accuracy (for instance 2 pixels). In the final step we further reduce our search area and search with final precision (for instance 1 pixel accuracy).<br><br>
Observe that this approach assumes, that the error function is more or less smooth over the search area, which is often true for natural images.<br><br>

An often used approach to reduce the search complexity is the so-called nearest neighbor search. It uses the assumption, that motion vectors don’t change much between neighboring blocks. Here we chose as start of our search first the origin (no movement) and then the previous motion vector,<br>

![Lecture6b-4.PNG](Img-Lecture6b\Lecture6b-4.PNG)
<br>

Another fast efficient approach to find the motion vectors is to **use the FFT** to compute a correlation function between shifted blocks, see e.g.:<br> 
S. Drew Kilthau and T. M. S. Moller, “Full Search Content Independent Block Matching Based on the Fast Fourier Transform”,  International Conference on Image Processing (ICIP) 2002, pp. I–669– I–672. <br><br>

In usual coders we transmit the found motion vectors to the decoder, as **side information**. In this way, the motion vectors are based on the current frame and the previous frame, and have the highest accuracy. The transmission of the motion vectors needs bit rate, but usually this is more than compensated for by the savings we get from better prediction and hence smaller difference values. In principle we could avoid sending motion vectors if we only base them on past frames, for instance the last frame and the frame before the last frame. In this case the decoder could perform the same operations to obtain the same motion vectors as the encoder, **without transmitting** them. But then the motion vectors would be **less accurate**, because they are not based on the current frame but on the previous frame. It would only work sufficiently if the motion does not change much.

</font></p>    

<font size="8" color ="Brown"><center>
## Python Example
        
<center></font>
<br>

<p style="line-height:1.5">
<font size="6">The following shows a python example for motion estimation with a full search in the range of +/- 8 pixels around the current block (which is a rather small range):<br>

```python videoencframewkYCrCb420mc.py```
<br>
</font></p>    

In [1]:
import numpy as np
import cv2
import sys 

if sys.version_info[0] < 3:
   # for Python 2
   import cPickle as pickle
else:
   # for Python 3
   import pickle

import scipy.signal

#Program to capture video from a camera and store it in an recording file, in Python txt format, using cPickle
# Also, filed is divided into luminance and two color components. By using 4:2:0 scheme, color components are then downsampled by the factor of 2. To reduce aliasing artifacts pyramidial filter is used.
#With motion estimation and display of motion vectors
#This is a framework for a simple video encoder to build.
#It writes into file 'videorecord.txt'
#Gerald Schuller, May 2015

cap = cv2.VideoCapture(0)
N=2
g=open('videorecord.txt', 'wb')
# filter is created by convolving two rectangular filters
#Triangular filter kernel:
filt1=np.ones((N,N))/N;
filt2=scipy.signal.convolve2d(filt1,filt1)/N

#Get size of frame:
[retval, frame] = cap.read()
[rows,columns,d]=frame.shape
print(rows,columns)
#Grid of 8x8 blocks:
gc=np.zeros((1,columns))
gc[0,0:columns:8]=np.ones(int(columns/8))
gr=np.zeros((rows,1))
gr[0:rows:8,0]=np.ones(int(rows/8))
grid=np.ones((rows,1))*gc+gr*np.ones((1,columns))
grid3=np.zeros((rows,columns,3))
grid3[:,:,1]=grid
#print(grid[0:9,0:9])


#Prevous Y frame:
Yprev=np.zeros((rows,columns))
#Vectors for current frame as graphic:
framevectors=np.zeros((rows,columns,3))
#motion vectors, for each block a 2-d vector:
mv=np.zeros((int(rows/8),int(columns/8),2))

#Process 25 frames:
for n in range(25):
    print("Frame no ",n)
    ret, frame = cap.read()
    [rows,columns,c]=frame.shape
 
    if ret==True:
       
        #Here goes the processing to reduce data... 
        reduced = np.zeros((rows,columns,c))
        Y=(0.114*frame[:,:,0]+0.587*frame[:,:,1]+0.299*frame[:,:,2]);        
        
        Cb=(0.4997*frame[:,:,0]-0.33107*frame[:,:,1]-0.16864*frame[:,:,2]);
   
        Cr=(-0.081282*frame[:,:,0]-0.418531*frame[:,:,1]+0.499813*frame[:,:,2]);
        reduced[:,:,0]=Y
        reduced[:,:,1]=Cb
        reduced[:,:,2]=Cr
       
        #print(grid3.shape)
        #print(framevectors.shape)

        cv2.imshow('Original',frame/255.0+framevectors)
        #cv2.imshow('Luminanz Y',Y[0:200,0:100]/255)
        #cv2.imshow('Unterabgetastetes Y',Ds)
        #cv2.imshow('Farbkomponente U',np.abs(Cr))
        #cv2.imshow('Farbkomponente V',np.abs(Cb))
       
   
        #Two color components are filtered first
        Crfilt=scipy.signal.convolve2d(Cr,filt2,mode='same')
        Cbfilt=scipy.signal.convolve2d(Cb,filt2,mode='same')

        # Downsampling
        DCr=Crfilt[0::N,::N];
        DCb=Cbfilt[0::N,::N];
        #cv2.imshow('Crfiltered',DCr)
        #cv2.imshow('Cbfiltered',DCb)

        #Motion estimation, correlate current Y block with previous 16x16 block which contains current 8x8 block:
        #Start pixel for block wise motion estimation:
        block=np.array([8,8])
        framevectors=np.zeros((rows,columns,3))
        #for loops for the blocks:
        #print("for loops for the motion vectors:")
        for yblock in range(5):
           #print("yblock=",yblock)
           block[0]=yblock*8+200;
           for xblock in range(5):
              #print("xblock=",xblock)
              block[1]=xblock*8+300;
              #print("block= ", block)
              #current block:
              Yc=Y[block[0]:block[0]+8 ,block[1]:block[1]+8]
              #print("Yc= ",Yc)
              #previous block:
              #Yp=Yprev[block[0]-4 : block[0]+12 ,block[1]-4 : block[1]+12]
              #print("Yp= ",Yp)
              #correlate both to find motion vector
              #print("Yp=",Yp)
              #print(Yc.shape)
              #Some high value for MAE for initialization:
              bestmae=10000.0;
              #For loops for the motion vector, full search at +-8 integer pixels:
              for ymv in range(-8,8):
                 for xmv in range(-8,8):
                    diff=Yc-Yprev[block[0]+ymv : block[0]+ymv+8, block[1]+xmv: block[1]+xmv+8];
                    mae=sum(sum(np.abs(diff)))/64;
                    if mae< bestmae:
                       bestmae=mae;
                       mv[yblock,xblock,0]=ymv;
                       mv[yblock,xblock,1]=xmv;

              #Ycorr=scipy.signal.correlate2d(Yp, Yc,mode='valid')
              #print("Ycorr= ", Ycorr)
              #motion vector:
              #1-d Index of maximum
              #index1d=np.argmax(Ycorr)
              #print("inedx1d=",index1d)
              #convert it to 2d index:
              #index2d=np.unravel_index(index1d,(9,9))
              #print("arg max of correlation: ", index2d)
              #2-d index minus center coordinates (4,4) is the motion vector
              #print("mv=",mv[0,0,:])
              #print(np.subtract(index2d,(4,4)))
              #mv[0,0,:]=np.subtract(index2d,(4,4))
              #print("mv[0,0,:]=",mv[0,0,:])
              #print(tuple(np.add(block,mv[0,0,:]).astype(int)))
              #cv2.line(framevectors, block, (block[0]+mv[0],block[1]+mv[1]),(1.0,1.0,1.0))
              cv2.line(framevectors, (block[1], block[0]), (block[1]+mv[yblock,yblock,1].astype(int),block[0]+mv[yblock,yblock,0].astype(int)) , (1.0,1.0,1.0));
        Yprev=Y.copy();
        #converting  images back to integer:
        Y=np.array(Y, dtype='uint8')
        DCr=np.array(DCr, dtype='int8')
        DCb=np.array(DCb, dtype='int8')
 #"Serialize" the captured video frame (convert it to a string) 
        #using pickle, and write/append it to file g:
        pickle.dump(Y,g)
        pickle.dump(DCr,g)
        pickle.dump(DCb,g)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
g.close()
cv2.destroyAllWindows()


480 640
Frame no  0
Frame no  1
Frame no  2
Frame no  3
Frame no  4
Frame no  5
Frame no  6
Frame no  7
Frame no  8
Frame no  9
Frame no  10
Frame no  11
Frame no  12
Frame no  13
Frame no  14
Frame no  15
Frame no  16
Frame no  17
Frame no  18
Frame no  19
Frame no  20
Frame no  21
Frame no  22
Frame no  23
Frame no  24


<p style="line-height:1.5">
<font size="6">
Observe: only 5x5 current blocks are chosen for motion estimation, and it is still quite slow. This shows that the faster approaches above are really necessary.
<br>
</font></p>    


<font size="8" color ="Brown"><center>
## Sub-Pixel Motion Estimation
        
<center></font>
<br>

<p style="line-height:1.5">
<font size="6">
    
Recent video coding standards get much of their improved compression from sub-pixel motion estimation. The basic approach is to increase the size of our search image using interpolation between pixels, and then search for the motion vector in this bigger image. For instance, if we want half pixel accuracy, we need to increase the number of pixels by 2, each horizontally and vertically (for a total factor of 4).<br><br>
An often used interpolation is the **“bi-linear”** interpolation. It computes the new pixel value simply as the **weighted average** of the values of the neighboring pixel values. The weighting is done using    <br>
![Lecture6b-5.PNG](Img-Lecture6b\Lecture6b-5.PNG)
<br>
Hence, if the new pixel is in the middle between two old pixels, the new pixel value (e.g. brightness) is simply the average of the values of the 2 neighbouring values. Or if the new pixel is in the middle (center) of 4 old pixels, its value is the average of those 4 pixel values. If the new pixel is closer to one of the old pixels, its value is the weighted average of the neighboring pixels, with an accordingly **higher weight for the closer pixel**.<br><br>
**Example:**
(from: http://en.wikipedia.org/wiki/Bilinear_interpolation)


![Lecture6b-6.PNG](Img-Lecture6b\Lecture6b-6.PNG)
<br>
</font></p>    